<a href="https://colab.research.google.com/github/iam-fern/Gen_datanew/blob/main/GQtest_cluster_loop_gendatanew_n30_x4_1000rep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Gen_datanew/raw/main/gendatanew_n30_x4_1000rep.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    32.871446  72.580450  ... -471664.983868 -2.866929e+05
      1.0    22.437717  64.959775  ...  437408.745566 -5.990445e+05
      2.0    76.202294  38.642559  ... -330941.315591  5.642280e+07
      3.0    81.671023  62.049919  ... -475797.984516 -4.043507e+07
      4.0    95.025609  67.101039  ... -424270.767464 -1.055856e+08
...                ...        ...  ...            ...           ...
999.0 25.0   81.856054  79.666499  ... -315740.292329 -3.978464e+07
      26.0   99.783219  62.704274  ... -336597.534805 -4.586394e+07
      27.0   66.556791  23.623048  ...  632757.776939  2.358060e+07
      28.0   76.825265  80.613707  ... -264865.866341 -2.334886e+07
      29.0   36.475837  61.439676  ... -260081.672523  2.124297e+06

[30000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    32.871446  72.580450  ... -471664.983868 -2.866929e+05
      1.0    22.437717  64.959775  ...  437408.745566 -5.990445e+05
      2.0    76.202294  38.642559  ... -330941.315591  5.642280e+07
      3.0    81.671023  62.049919  ... -475797.984516 -4.043507e+07
      4.0    95.025609  67.101039  ... -424270.767464 -1.055856e+08
...                ...        ...  ...            ...           ...
999.0 25.0   81.856054  79.666499  ... -315740.292329 -3.978464e+07
      26.0   99.783219  62.704274  ... -336597.534805 -4.586394e+07
      27.0   66.556791  23.623048  ...  632757.776939  2.358060e+07
      28.0   76.825265  80.613707  ... -264865.866341 -2.334886e+07
      29.0   36.475837  61.439676  ... -260081.672523  2.124297e+06

[30000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,32.871446,72.580450,29.847440,-7.585032e+05,-7.583579e+05,-471664.983868,-2.866929e+05
1,0.0,1.0,22.437717,64.959775,99.086252,-1.618322e+05,-1.616357e+05,437408.745566,-5.990445e+05
2,0.0,2.0,76.202294,38.642559,29.114222,5.609170e+07,5.609186e+07,-330941.315591,5.642280e+07
3,0.0,3.0,81.671023,62.049919,42.335798,-4.091106e+07,-4.091087e+07,-475797.984516,-4.043507e+07
4,0.0,4.0,95.025609,67.101039,58.515686,-1.060101e+08,-1.060099e+08,-424270.767464,-1.055856e+08
...,...,...,...,...,...,...,...,...,...
29995,999.0,25.0,81.856054,79.666499,74.653742,-4.010062e+07,-4.010038e+07,-315740.292329,-3.978464e+07
29996,999.0,26.0,99.783219,62.704274,64.269378,-4.620077e+07,-4.620053e+07,-336597.534805,-4.586394e+07
29997,999.0,27.0,66.556791,23.623048,96.182635,2.421316e+07,2.421336e+07,632757.776939,2.358060e+07
29998,999.0,28.0,76.825265,80.613707,77.774931,-2.361398e+07,-2.361373e+07,-264865.866341,-2.334886e+07


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    32.871446  72.580450  ... -471664.983868 -2.866929e+05
      1.0    22.437717  64.959775  ...  437408.745566 -5.990445e+05
      2.0    76.202294  38.642559  ... -330941.315591  5.642280e+07
      3.0    81.671023  62.049919  ... -475797.984516 -4.043507e+07
      4.0    95.025609  67.101039  ... -424270.767464 -1.055856e+08
...                ...        ...  ...            ...           ...
999.0 25.0   81.856054  79.666499  ... -315740.292329 -3.978464e+07
      26.0   99.783219  62.704274  ... -336597.534805 -4.586394e+07
      27.0   66.556791  23.623048  ...  632757.776939  2.358060e+07
      28.0   76.825265  80.613707  ... -264865.866341 -2.334886e+07
      29.0   36.475837  61.439676  ... -260081.672523  2.124297e+06

[30000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)
    data_sort_clusters = data_clusters.sort_values('clusters')
    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid + 1
    SSR1 = model1.ssr
    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid + 1
    SSR0 = model0.ssr
    MSE0 = model0.mse_resid

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/DF1) / (SSR0/DF0)

    df1 = DF1
    df0 = DF0

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,8.0,16.0,7815567408437888.0,2.105061056066254e+16,1116509629776841.1,1403374037377502.8,1.3467103193260004,0.6903749188873123,0.6548125405563439,Reject005=1 : Homoscedasticity
3.0,7.0,17.0,2842379340999727.0,1.0480525120980734e+16,473729890166621.2,655032820061295.9,1.518273891765405,0.5934080147083671,0.7032959926458164,Reject005=1 : Homoscedasticity
5.0,8.0,16.0,1530855814173173.5,1.3516530076127272e+16,218693687739024.78,901102005075151.5,4.414697305581209,0.039303621584824366,0.9803481892075878,Reject005=0 : Heteroscedasticity
7.0,14.0,10.0,1758628595952531.5,3.728871300012083e+16,135279122765579.34,4143190333346759.0,29.68460669883151,2.1127461047321106e-07,0.9999998943626948,Reject005=0 : Heteroscedasticity
8.0,15.0,9.0,3741960558227002.5,5.0570961351013336e+16,267282897016214.47,6321370168876667.0,22.52427150781025,7.596228859263476e-07,0.999999620188557,Reject005=0 : Heteroscedasticity
9.0,12.0,12.0,1.0563274274965956e+16,1.0945936585278444e+16,960297661360541.5,995085144116222.1,1.036225728912423,0.9518656165774122,0.5240671917112939,Reject005=1 : Homoscedasticity
11.0,6.0,18.0,70068585204050.836,6.138938507144409e+16,14013717040810.168,3611140298320240.5,292.04426345353136,4.856757780391519e-07,0.999999757162111,Reject005=0 : Heteroscedasticity
12.0,8.0,16.0,134096814120160.94,8.031820274896707e+16,19156687731451.562,5354546849931138.0,299.4784151881336,5.04670816248165e-09,0.9999999974766459,Reject005=0 : Heteroscedasticity
16.0,12.0,12.0,106174767844074.42,1.723952759557295e+16,9652251622188.584,1567229781415722.8,162.36934580248374,4.7339909770016675e-11,0.99999999997633,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,15.0,9.0,5.22068251217068e+16,8241552352078.638,3729058937264771.5,1030194044009.8297,0.0002631058401803971,6.350251035817131e-15,Reject005=0 : Heteroscedasticity
1.0,8.0,16.0,7815567408437888.0,2.105061056066254e+16,1116509629776841.1,1403374037377502.8,1.3467103193260004,1.3096250811126877,Reject005=1 : Homoscedasticity
2.0,13.0,11.0,2.5563580685697e+16,580409200988258.6,2130298390474750.0,58040920098825.86,0.02683263174498378,6.974145477551208e-07,Reject005=0 : Heteroscedasticity
3.0,7.0,17.0,2842379340999727.0,1.0480525120980734e+16,473729890166621.2,655032820061295.9,1.518273891765405,1.4065919852916329,Reject005=1 : Homoscedasticity
4.0,10.0,14.0,3.912566379819008e+16,3734308316280189.0,4347295977576675.5,287254485867706.9,0.06817425761299589,1.8330669411370617e-05,Reject005=0 : Heteroscedasticity
5.0,8.0,16.0,1530855814173173.5,1.3516530076127272e+16,218693687739024.78,901102005075151.5,4.414697305581209,0.039303621584824366,Reject005=0 : Heteroscedasticity
6.0,11.0,13.0,1.8506676330981708e+16,1.6972301597976796e+16,1850667633098170.8,1414358466498066.2,0.7759998617995677,0.6556728575823324,Reject005=1 : Homoscedasticity
7.0,14.0,10.0,1758628595952531.5,3.728871300012083e+16,135279122765579.34,4143190333346759.0,29.68460669883151,2.1127461047321106e-07,Reject005=0 : Heteroscedasticity
8.0,15.0,9.0,3741960558227002.5,5.0570961351013336e+16,267282897016214.47,6321370168876667.0,22.52427150781025,7.596228859263476e-07,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    673
Reject005=1 : Homoscedasticity      327
Name: Result_test, dtype: int64